In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import ast
data_clean = pd.read_csv('/content/drive/MyDrive/Bahan Penelitian/dataset penelitian/data_bersih.csv')

# Mengubah string menjadi list
data_clean['isi_stemmed'] = data_clean['isi_stemmed'].apply(ast.literal_eval)

In [3]:
dictionary =Dictionary(data_clean['isi_stemmed'])
corpus =[dictionary.doc2bow(text) for text in data_clean['isi_stemmed']]

In [4]:
from gensim.models import CoherenceModel
def compute_coherence_values(corpus, dictionary, k, a, b):
  lda_model = LdaModel(corpus=corpus,
                       id2word=dictionary,
                       num_topics=k,
                       chunksize=100,
                       passes=10,
                       iterations=100,
                       alpha=a,
                       eta=b,
                       random_state=42)
  coherence_model_lda =CoherenceModel(model=lda_model,texts=data_clean['isi_stemmed'],dictionary=dictionary,coherence='c_v')
  coherence_score = coherence_model_lda.get_coherence()
  return coherence_score

In [5]:
import tqdm

# Topics range
min_topics =11
max_topics =14
step_size =1
topics_range =range(min_topics,max_topics,step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.1))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.1))
beta.append('symmetric')

model_results ={'Topics':[],
                'Alpha':[],
                'Beta':[],
                'Coherence':[]
                }
if 1 == 1:
  pbar =tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)))

  for k in topics_range:
    for a in alpha:
      for b in beta:
        cv = compute_coherence_values(corpus=corpus,dictionary=dictionary,
                                        k=k, a=a, b=b)
        model_results['Topics'].append(k)
        model_results['Alpha'].append(a)
        model_results['Beta'].append(b)
        model_results['Coherence'].append(cv)

        pbar.update(1)
  pd.DataFrame(model_results).to_csv('/content/drive/MyDrive/Bahan Penelitian/dataset penelitian/tunning_coherence4.csv',index=False)
  pbar.close()


100%|██████████| 396/396 [11:02:42<00:00, 100.41s/it]


# bla bla

In [ ]:
# Inisialisasi variabel
num_topics_list = range(2,15)  # Jumlah topik yang akan diuji
perplexity_scores = []
beta ='symmetric'
# Melatih model LDA dan menghitung perplexity untuk setiap jumlah topik
for num_topics in num_topics_list:
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics,alpha='symmetric',eta=beta,passes=10, iterations=100, random_state=42 )
    perplexity = lda_model.log_perplexity(corpus)
    perplexity_scores.append(perplexity)

# Plot jumlah topik versus perplexity
plt.plot(num_topics_list, perplexity_scores, '-o')
plt.xlabel('Jumlah Topik')
plt.ylabel('Perplexity')
plt.title('Jumlah Topik versus Perplexity')
plt.grid(True)
plt.show()

In [ ]:
# Fungsi untuk menghitung perplexity
def compute_perplexity(dictionary, corpus, num_topics, alpha, beta):
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, alpha=alpha, eta=beta, passes=10, iterations=100, random_state=42 )
    perplexity = lda_model.log_perplexity(corpus)
    return perplexity

# Penyetelan jumlah topik, alpha, dan beta
num_topics_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
alpha_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
beta_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

best_perplexity = float('inf')
best_num_topics = None
best_alpha = None
best_beta = None

# Loop untuk mencari kombinasi nilai jumlah topik, alpha, dan beta dengan perplexity terendah
for num_topics in num_topics_list:
    for alpha in alpha_list:
        for beta in beta_list:
            perplexity = compute_perplexity(dictionary, corpus, num_topics, alpha, beta)
            if perplexity < best_perplexity:
                best_perplexity = perplexity
                best_num_topics = num_topics
                best_alpha = alpha
                best_beta = beta

# Menampilkan hasil
print("Jumlah Topik terbaik:", best_num_topics)
print("Alpha terbaik:", best_alpha)
print("Beta terbaik:", best_beta)
print("Perplexity terendah:", best_perplexity)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Jumlah Topik terbaik: 5
Alpha terbaik: 0.1
Beta terbaik: 0.1
Perplexity terendah: -7.873277982542808
